In [1]:
import numpy as np
import torch
import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys
import pickle
from torch.optim import SGD

In [2]:
sys.path.insert(1, '/home/ubuntu/Intents-Analysis/Analysis')
#sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [3]:
from get_vocab import load_data, get_vocab
from get_frequency import get_frequency

In [4]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)
CUDA_LAUNCH_BLOCKING=1

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is True
8


In [5]:
##Needed Functions
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output


def create_vocabulary(train_file):
    '''This function creates an indexed vocabulary dictionary from the training file'''
    
    vocab, _ = get_vocab(1, train_file)
    
    phone_to_idx = {'unk': 1}#Padding indx = 0, unkown_idx = 1, indexing starts from 2
    for i, phone in enumerate(vocab):
        phone_to_idx[phone] = i + 2
        
    return phone_to_idx

In [6]:
class MyDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                if len(utterance) != 0:
                    utterance_to_idx = []

                    for phone in utterance:
                        if phone not in phone_to_idx:
                            phone = 'unk'

                        utterance_to_idx.append(phone_to_idx[phone])

                    self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        label = self.all_data[index][1]

        return input_vector, label

In [7]:
def collate_indic(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)

    # collate y
    y = torch.tensor(y_lst)

    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [8]:
def get_domains():
    all_intents = ['increase', 'decrease', 'activate', 'deactivate', 'bring', 'change language']
    return all_intents

def get_intents():
    all_intents = [
        'activate|lamp',
        'activate|lights|bedroom',
        'activate|lights|kitchen',
        'activate|lights|none',
        'activate|lights|washroom',
        'activate|music',
        'bring|juice',
        'bring|newspaper',
        'bring|shoes',
        'bring|socks',
        'change language|Chinese',
        'change language|English',
        'change language|German',
        'change language|Korean',
        'change language|none',
        'deactivate|lamp',
        'deactivate|lights|bedroom',
        'deactivate|lights|kitchen',
        'deactivate|lights|none',
        'deactivate|lights|washroom',
        'deactivate|music',
        'decrease|heat|bedroom',
        'decrease|heat|kitchen',
        'decrease|heat|none',
        'decrease|heat|washroom',
        'decrease|volume',
        'increase|heat|bedroom',
        'increase|heat|kitchen',
        'increase|heat|none',
        'increase|heat|washroom',
        'increase|volume'
        ]

    return all_intents

def get_intent_labels(class_type):
    if class_type == 'domain':
        all_intents = get_domains()
    else:
        all_intents = get_intents()
        
    intent_labels = {}
    labels_to_intents = {}
    for i, intent in enumerate(all_intents):
        intent_labels[intent] = i
        labels_to_intents[i] = intent
        
    return intent_labels, labels_to_intents

In [9]:
class_type = 'intents'
split = 'train'

intent_labels, labels_to_intents = get_intent_labels(class_type)

#Loading data
train_file = '../FSC/fsc_' + class_type + '_' + split + '.pkl'
test_file = '../FSC/fsc_' + class_type + '_test.pkl'
#create vocabulary and phone_to_idx
phone_to_idx = create_vocabulary(train_file)
print(len(phone_to_idx))

49


In [10]:
train_dataset = MyDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_indic)

test_dataset = MyDataset(test_file, intent_labels, phone_to_idx)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
valid_loader = DataLoader(test_dataset, **test_loader_args, collate_fn=collate_indic)

In [36]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size=50, embed_size=128, hidden_size=384, label_size=31):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(embed_size*3)

        self.lstm = nn.LSTM(embed_size*3, hidden_size, num_layers=2)
        self.linear = nn.Linear(hidden_size, label_size)

    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """

        # B,T,H
        input = self.embed(x)

        # (B,T,H) -> (B,H,T)
        input = input.transpose(1,2)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        # (B,H,T)
        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)

        output, (hn, cn) = self.lstm(pack_tensor)

        #output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        #output = torch.cat([hn[0], hn[1]], dim=1)
        logits = self.linear(hn[0])

        return logits

In [39]:
model = RNNClassifier()
opt = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.CrossEntropyLoss()
#opt = SGD(model.parameters(), lr=0.05)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

RNNClassifier(
  (embed): Embedding(50, 128)
  (cnn): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (cnn3): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
  (batchnorm): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(384, 384, num_layers=2)
  (linear): Linear(in_features=384, out_features=31, bias=True)
)

In [ ]:
print(class_type, split)
max_acc = 0

for j in range(1000):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0
    err_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()

        logits = model(x, lengths)

        loss = criterion(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1
                    

    print("train acc: ", acc_cnt/(err_cnt+acc_cnt), " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

    model.eval()
    acc_cnt = 0
    err_cnt = 0

    #start_time = time.time()
    for x, lengths, y in valid_loader:
        
        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        
        logits = model(x, lengths)

        out_val, out_indices = torch.max(logits, dim=1)
        tar_indices = y

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    if current_acc > max_acc:
        max_acc = current_acc
                
    print(j, "validation: ", current_acc, '--max', max_acc, '--time:', time.time() - start_time)

intents train
train acc:  0.5656762250767701  train loss:  1.3870387353949785 --time: 27.709045886993408
0 validation:  0.8072257383966245 --max 0.8072257383966245 --time: 31.27848243713379
train acc:  0.8185199602093335  train loss:  0.5749109726584418 --time: 27.42573571205139
1 validation:  0.8847573839662447 --max 0.8847573839662447 --time: 31.353657245635986
train acc:  0.8782924613987284  train loss:  0.38989038246771246 --time: 27.913216590881348
2 validation:  0.897415611814346 --max 0.897415611814346 --time: 31.967920780181885
train acc:  0.9080489598200769  train loss:  0.29097310729448306 --time: 27.90494656562805
3 validation:  0.9013713080168776 --max 0.9013713080168776 --time: 31.91893482208252
train acc:  0.9300635785649409  train loss:  0.21519309422258515 --time: 27.770965576171875
4 validation:  0.9071729957805907 --max 0.9071729957805907 --time: 31.782634496688843
train acc:  0.946152848060205  train loss:  0.1704964364431181 --time: 27.881584644317627
5 validation: 

train acc:  0.9983564724709139  train loss:  0.003920862024004548 --time: 28.306659698486328
46 validation:  0.9195675105485233 --max 0.9240506329113924 --time: 31.9681715965271
train acc:  0.9982699710220146  train loss:  0.0043623430048371535 --time: 28.346237897872925
47 validation:  0.9190400843881856 --max 0.9240506329113924 --time: 32.03111672401428
train acc:  0.9984862246442628  train loss:  0.004595090844395097 --time: 28.21490240097046
48 validation:  0.9200949367088608 --max 0.9240506329113924 --time: 31.919862747192383
train acc:  0.9981834695731153  train loss:  0.004767956597024951 --time: 28.165061473846436
49 validation:  0.9179852320675106 --max 0.9240506329113924 --time: 31.91321635246277
train acc:  0.9592578175684443  train loss:  0.12270525450952022 --time: 28.247945070266724
50 validation:  0.9037447257383966 --max 0.9240506329113924 --time: 31.975516319274902
train acc:  0.9801046667531681  train loss:  0.06271103906409187 --time: 28.419225931167603
51 validation

train acc:  0.9985294753687124  train loss:  0.003564983304808346 --time: 27.632081985473633
92 validation:  0.9208860759493671 --max 0.9240506329113924 --time: 32.00350332260132
train acc:  0.998572726093162  train loss:  0.0033651491508631674 --time: 27.812506437301636
93 validation:  0.9216772151898734 --max 0.9240506329113924 --time: 32.11572766304016
train acc:  0.9982699710220146  train loss:  0.0037728226723539185 --time: 27.739736557006836
94 validation:  0.9206223628691983 --max 0.9240506329113924 --time: 32.096752643585205
train acc:  0.9983132217464643  train loss:  0.0036565493952688114 --time: 27.590723991394043
95 validation:  0.9219409282700421 --max 0.9240506329113924 --time: 31.94054079055786
train acc:  0.9983997231953635  train loss:  0.0038754207729733386 --time: 27.807960033416748
96 validation:  0.9203586497890295 --max 0.9240506329113924 --time: 32.05123257637024
train acc:  0.9982699710220146  train loss:  0.004194337664633052 --time: 27.661723613739014
97 valid

In [ ]:
2 LSTM layer, size 384